### EXTRAER DATOS Api SPOTIFY

In [2]:
import requests
import base64
import pandas as pd
import mysql.connector


# Credenciales de tu app
CLIENT_ID = 'a80cf890990b4f3e9585bb385be538d0'
CLIENT_SECRET = '6445dff7ddd44f49a4c98f3da475a2f4'

# URL para obtener el token
url = 'https://accounts.spotify.com/api/token'

# Codifica tus credenciales
credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# Encabezados y datos
headers = {
    'Authorization': f'Basic {encoded_credentials}',
}
data = {
    'grant_type': 'client_credentials',
}
# Solicitar el token
response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
    token = response.json().get('access_token')
    print(f'Access Token: {token}')
else:
    print(f'Error: {response.status_code}')
    print(response.json())

Access Token: BQDX40lJ-QnpISEtTAl1Q8NTLF45YKcH8nopABLIxwCO_PFnjMtWnoLvUiF0yeD45wlrZQT53UZcP7prHMl26qBnK2CrLbahVAto76buSl-kQRlAJnc


In [22]:

# Token de acceso que obtuviste previamente
access_token = "BQABfKP8-nmGSI5L9nofAsSc69gK1_VR3E5zo9VaRjEc9m7FkVV6x-uUO3gu-nqD4bgNJsSlbdLAJ33WQvQumsKaI3fIPZGQtxV_aExSvrJkrxEhIQ4"

# Encabezados con el token de acceso
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Géneros y rango de años
generos = ["pop", "rock", "jazz", "hip-hop"]
años = range(2019, 2024)  # De 2019 a 2023

# Esta función va a hacer la consulta y extraer la información
def obtener_info_spotify(generos, años):
    resultados = []

    for genero in generos:
        for año in años:
            print(f"Buscando {genero} del año {año}...")
            url = "https://api.spotify.com/v1/search"
            params = {
                "q": f"genre:{genero} year:{año}",  # Filtrar por género y año
                "type": "album",  # Puedes cambiarlo a 'track' para canciones
                "limit": 50  # Número de resultados
            }

            # Hacemos la solicitud a la API
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                print(f"Resultados para {genero} del año {año}: {len(data.get('albums', {}).get('items', []))}")  # Imprime la cantidad de resultados obtenidos


                # Recorremos los resultados
                for item in data.get('albums', {}).get('items', []):
                    album_info = {
                        "artista": ", ".join([artist["name"] for artist in item["artists"]]),  # Nombre del artista(s)
                        "genero": genero,  # Género
                        "tipo": "album",  # Tipo de resultado (puede ser "track" o "album")
                        "nombre": item["name"],  # Nombre del álbum
                        "año_lanzamiento": item["release_date"],  # Año de lanzamiento
                        "id": item["id"],  # ID del álbum
                    }
                    resultados.append(album_info)

            else:
                print(f"Error en la solicitud: {response.status_code}")
                print(response.json())

    return resultados

# Llamamos a la función para obtener los resultados
resultados = obtener_info_spotify(generos, años)

# Convertimos los resultados en un DataFrame de pandas
df = pd.DataFrame(resultados)

# Verificamos los primeros resultados
print(df.head())

# Conexión a la base de datos MySQL
db_connection = mysql.connector.connect(
    host="127.0.0.1",   # Host de la base de datos (usualmente 'localhost')
    port= "3306",      # Port de la base de datos 
    user="root",           # Usuario de la base de datos
    password="AlumnaAdalab",  # Contraseña de la base de datos
    database="MusicStream"  # Nombre de tu base de datos
)

# Crear un cursor para ejecutar queries
cursor = db_connection.cursor()

# Crear la tabla en MySQL (si no existe)

cursor.execute('CREATE TABLE IF NOT EXISTS albums(id VARCHAR(255) PRIMARY KEY, artista TEXT, genero VARCHAR(255), tipo VARCHAR(50), nombre VARCHAR(255), año_lanzamiento DATE)')

# Convertir la columna 'año_lanzamiento' a tipo datetime
df['año_lanzamiento'] = pd.to_datetime(df['año_lanzamiento'], errors='coerce')

# O si solo deseas insertar el año (sin fecha completa)
df['año_lanzamiento'] = df['año_lanzamiento'].dt.year

# Insertamos los datos en la base de datos, verificando si el ID ya existe
for _, row in df.iterrows():
    cursor.execute("SELECT COUNT(*) FROM albums WHERE id = %s", (row['id'],))
    result = cursor.fetchone()
    
    # Si el id no existe, insertamos el nuevo registro
    if result[0] == 0:
        cursor.execute("""
            INSERT INTO albums (id, artista, genero, tipo, nombre, año_lanzamiento)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['id'], row['artista'], row['genero'], row['tipo'], row['nombre'], row['año_lanzamiento']))

# Confirmar los cambios
db_connection.commit()


# Confirmar los cambios
db_connection.commit()

# Confirmamos que los datos se han almacenado correctamente
print("Datos almacenados en la base de datos MySQL con éxito.")

# Cerramos la conexión
cursor.close()
db_connection.close()


Buscando pop del año 2019...
Resultados para pop del año 2019: 2
Buscando pop del año 2020...
Resultados para pop del año 2020: 0
Buscando pop del año 2021...
Resultados para pop del año 2021: 1
Buscando pop del año 2022...
Resultados para pop del año 2022: 0
Buscando pop del año 2023...
Resultados para pop del año 2023: 0
Buscando rock del año 2019...
Resultados para rock del año 2019: 1
Buscando rock del año 2020...
Resultados para rock del año 2020: 2
Buscando rock del año 2021...
Resultados para rock del año 2021: 0
Buscando rock del año 2022...
Resultados para rock del año 2022: 1
Buscando rock del año 2023...
Resultados para rock del año 2023: 1
Buscando jazz del año 2019...
Resultados para jazz del año 2019: 2
Buscando jazz del año 2020...
Resultados para jazz del año 2020: 8
Buscando jazz del año 2021...
Resultados para jazz del año 2021: 3
Buscando jazz del año 2022...
Resultados para jazz del año 2022: 0
Buscando jazz del año 2023...
Resultados para jazz del año 2023: 0
Busca

In [23]:
print(df)

                                              artista   genero   tipo  \
0                                          André Popp      pop  album   
1                                    Typical Minority      pop  album   
2                                         Pattawiguys      pop  album   
3                                  The Tower of Dudes     rock  album   
4                                           Scenester     rock  album   
5            Ibiza DJ Rockerz, Crazy Party Music Guys     rock  album   
6                                               Genre     rock  album   
7                                          Dj ROCKENT     rock  album   
8                                         Onda Alegre     jazz  album   
9   Instrumental Jazz Music Guys, Instrumental, Re...     jazz  album   
10                                     Alphagarrsongs     jazz  album   
11                          Jazz Night Music Paradise     jazz  album   
12  Jazz Instrumentals, Jazz Music Lovers Club, Co.

In [4]:
# EXTRACCIÓN CANCIONES DE SPOTIFY 

import requests
import pandas as pd
from sqlalchemy import create_engine

# Token de acceso que obtuviste previamente
access_token = "BQDX40lJ-QnpISEtTAl1Q8NTLF45YKcH8nopABLIxwCO_PFnjMtWnoLvUiF0yeD45wlrZQT53UZcP7prHMl26qBnK2CrLbahVAto76buSl-kQRlAJnc"

# Encabezados con el token de acceso
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Géneros y rango de años
generos = ["pop", "rock", "jazz", "hip-hop"]
años = range(2019, 2024)  # De 2019 a 2023

# Esta función va a hacer la consulta y extraer la información
def obtener_info_spotify(generos, años):
    resultados = []

    for genero in generos:
        for año in años:
            print(f"Buscando {genero} del año {año}...")
            url = "https://api.spotify.com/v1/search"
            params = {"q": f"year:{año}",  # Solo filtrar por año (puedes cambiar a género si lo prefieres)
                "type": "track",  # Tipo de búsqueda
                "limit": 50  # Número de resultados
            }

            # Hacemos la solicitud a la API
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                print(f"Resultados para {genero} del año {año}: {len(data.get('tracks', {}).get('items', []))}")  # Correcto: 'tracks'

                # Recorremos los resultados
                for item in data.get('tracks', {}).get('items', []):
                    cancion_info = {
                        "artista": ", ".join([artist["name"] for artist in item["artists"]]),  # Nombre del artista(s)
                        "genero": genero,  # Género
                        "tipo": "track",  # Tipo de resultado (puede ser "track" o "album")
                        "nombre": item["name"],  # Nombre de la canción
                        "album": item["album"]["name"],  # Nombre del álbum
                        "id": item["id"],  # ID de la canción
                    }
                    resultados.append(cancion_info)

            else:
                print(f"Error en la solicitud: {response.status_code}")
                print(response.json())

    return resultados

# Llamamos a la función para obtener los resultados
resultados = obtener_info_spotify(generos, años)

# Convertimos los resultados en un DataFrame de pandas
df = pd.DataFrame(resultados)

# Verificamos los primeros resultados
print(df.head())

# Configurar conexión a la base de datos MySQL
host = "127.0.0.1"  # Host de la base de datos (usualmente 'localhost')
port = "3306"  # Puerto de la base de datos
user = "root"  # Usuario de la base de datos
password = "AlumnaAdalab"  # Contraseña de la base de datos
database = "MusicStream"  # Nombre de tu base de datos

# Crear la cadena de conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# Pasar el DataFrame a la base de datos
df.to_sql('canciones', con=engine, if_exists='replace', index=False)




Buscando pop del año 2019...
Resultados para pop del año 2019: 50
Buscando pop del año 2020...
Resultados para pop del año 2020: 50
Buscando pop del año 2021...
Resultados para pop del año 2021: 50
Buscando pop del año 2022...
Resultados para pop del año 2022: 50
Buscando pop del año 2023...
Resultados para pop del año 2023: 50
Buscando rock del año 2019...
Resultados para rock del año 2019: 50
Buscando rock del año 2020...
Resultados para rock del año 2020: 50
Buscando rock del año 2021...
Resultados para rock del año 2021: 50
Buscando rock del año 2022...
Resultados para rock del año 2022: 50
Buscando rock del año 2023...
Resultados para rock del año 2023: 50
Buscando jazz del año 2019...
Resultados para jazz del año 2019: 50
Buscando jazz del año 2020...
Resultados para jazz del año 2020: 50
Buscando jazz del año 2021...
Resultados para jazz del año 2021: 50
Buscando jazz del año 2022...
Resultados para jazz del año 2022: 50
Buscando jazz del año 2023...
Resultados para jazz del año

-1

In [15]:
url_base = "http://ws.audioscrobbler.com/2.0/"
api_key = "9de743ec4e44e6ebfa1dd9a04931600d"

In [16]:
# Funcion para info LastFm 

def obtener_info_lastfm(nombre_artista):
    artista = {"nombre": "", "biografia": "", "oyentes": "", "reproducciones": "", "similares": []}
    url_base = "http://ws.audioscrobbler.com/2.0/"  # Construir la URL para la solicitud de información del artista
    url_artista = url_base + "?method=artist.getinfo&artist=" + nombre_artista + "&api_key=" + api_key + "&format=json"


    # Hacer la solicitud HTTP a la API
    response = requests.get(url_artista)
    
    if response.status_code == 200: #Si es correcto la solicitud ejecuta lo siguiente:
        data = response.json()
        artista['nombre']= data['artist']['name']
        artista['biografia']= data['artist']['bio']['summary']
        artista['oyentes'] = data['artist']['stats']['listeners']
        artista['reproducciones']= data['artist']['stats']['playcount']
        for artista_similar in data['artist']['similar']['artist']:
            artista['similares'].append(artista_similar['name'])
        return artista
    else:
        print(f"Error en la solicitud:{response.status_code}")
        print(response.json())


In [17]:
obtener_info_lastfm('André Popp')

{'nombre': 'André Popp',
 'biografia': 'Andre Popp contributed two of the greatest albums of the space age pop era, both released here on Columbia\'s short-lived but remarkable "World of Discovery" series: "Delirium in Hi-Fi" and "Popped!"\n\nThe son of a church organist, Popp studied at the St. Josephe Institute of Music and took over his father\'s position in 1939, when his father joined the French Army. After the war, he met Jean Broussole, a poet and lyricist, and together they moved to Paris, when they began to have success with songs such as "Il Dansait. <a href="https://www.last.fm/music/+noredirect/Andr%C3%A9+Popp">Read more on Last.fm</a>',
 'oyentes': '11806',
 'reproducciones': '96148',
 'similares': ['John Buzon Trio',
  'Dean Elliott & His Big Band',
  'Terry Snyder',
  'Denny McLain',
  'Leroy Holmes']}

In [19]:
# Funcion para info LastFm 

def obtener_info_lastfm(nombre_artista):
    artista = {"nombre": "", "biografia": "", "oyentes": "", "reproducciones": "", "similares": []}
    url_base = "http://ws.audioscrobbler.com/2.0/"  # Construir la URL para la solicitud de información del artista
    url_artista = url_base + "?method=artist.getinfo&artist=" + nombre_artista + "&api_key=" + api_key + "&format=json"


    # Hacer la solicitud HTTP a la API
    response = requests.get(url_artista)
    
    if response.status_code == 200: #Si es correcto la solicitud ejecuta lo siguiente:
        data = response.json()
        artista['nombre']= data['artist']['name']
        artista['biografia']= data['artist']['bio']['summary']
        artista['oyentes'] = data['artist']['stats']['listeners']
        artista['reproducciones']= data['artist']['stats']['playcount']
        for artista_similar in data['artist']['similar']['artist']:
            artista['similares'].append(artista_similar['name'])
        return artista
    else:
        print(f"Error en la solicitud:{response.status_code}")
        print(response.json())


# Conexión a la base de datos MySQL
db_connection = mysql.connector.connect(
    host="127.0.0.1",   # Host de la base de datos (usualmente 'localhost')
    port= "3306",      # Port de la base de datos 
    user="root",           # Usuario de la base de datos
    password="AlumnaAdalab",  # Contraseña de la base de datos
    database="MusicStream"  # Nombre de tu base de datos
)

# Crear un cursor para ejecutar queries
cursor = db_connection.cursor()

# Crear la tabla en MySQL (si no existe)
cursor.execute('CREATE TABLE IF NOT EXISTS last_fm (nombre VARCHAR(255) PRIMARY KEY, biografia VARCHAR(255), oyentes INT, reproducciones INT, similares VARCHAR(255))')


# Insertamos los datos en la base de datos, verificando si el nombre ya existe
for _, row in df.iterrows():
    cursor.execute("SELECT COUNT(*) FROM last_fm WHERE nombre = %s", (row['nombre'],))
    result = cursor.fetchone()
    
    # Si el nombre no existe, insertamos el nuevo registro
    if result[0] == 0:
        cursor.execute("""
            INSERT INTO last_fm (nombre, biografia, oyentes, reproducciones, similares)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['nombre'], row.get('biografia', 'No disponible'), row['oyentes'], row['reproducciones'], row['similares']))



# Confirmar los cambios
db_connection.commit()


# Confirmamos que los datos se han almacenado correctamente
print("Datos almacenados en la base de datos MySQL con éxito.")

# Cerramos la conexión
cursor.close()
db_connection.close()

KeyError: 'oyentes'